In [1]:
import paramiko

# SSH 접속 정보
ip = "172.16.24.111"
user = "root"
pw = "asd123!@"

def cc(command):
    """SSH로 명령을 실행하고 결과를 출력"""
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(command)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    if err:
        print("⚠️ 오류 발생:", err)
    return out.strip()

def install_tools():
    cmd = """
apt update -y
apt upgrade -y
apt install -y vim wget network-manager
"""
    cc(cmd)
    print("기본 툴 설치 완료")

def install_ssh():
    cmd = """
apt install -y openssh-server
systemctl enable ssh
systemctl start ssh
"""
    cc(cmd)
    print("SSH 설치 및 서비스 시작 완료")

def config_ufw():
    cmd = """
apt install -y ufw
ufw allow ssh
ufw --force enable
shutdown -r now
"""
    cc(cmd)
    print("UFW 설정 완료, 시스템 재부팅 중")

def go():
    install_ssh()
    config_ufw()

if __name__ == "__main__":
    go()


⚠️ 오류 발생: 

Synchronizing state of ssh.service with SysV service script with /usr/lib/systemd/systemd-sysv-install.
Executing: /usr/lib/systemd/systemd-sysv-install enable ssh
Created symlink /etc/systemd/system/sshd.service → /usr/lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /usr/lib/systemd/system/ssh.service.

SSH 설치 및 서비스 시작 완료
⚠️ 오류 발생: 


UFW 설정 완료, 시스템 재부팅 중


In [ ]:
import subprocess
import os
# 로컬에서는 이미 expect가 설치되어있어야합니다. 없으면 3번째 줄 주석하시고 local_set() 함수 실행하세요
# 그 후에 다시 주석 해제해주세요.
user = "root"
host = "172.16.16.25"
password = "asd123!@"

########################### 로컬 ###########################
## 우선 로컬에서 원격 호스트 IP에 대한 StrictHostKeyChecking=no를 항상 허용해줍니다.

def local_ssh_config():
    ssh_dir = os.path.expanduser("~/.ssh")
    config_path = os.path.join(ssh_dir,"config")
    
    os.makedirs(ssh_dir, exist_ok=True)
    os.chmod(ssh_dir, 0o700)
    
    config_content = f"""
    Host {host}
        StrictHostKeyChecking no
        UserKnownHostsFile=/dev/null
    """
    
    # 이미 같은 설정이 있는지 확인하고 없으면 추가
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            content = f.read()
    else:
        content = ""
    
    if host not in content:
        with open(config_path, "a") as f:
            f.write(config_content)
        print(f"✅ SSH 설정 추가 완료: {host}")
    else:
        print(f"ℹ️ 이미 설정되어 있습니다: {host}")
    
    # 권한 설정 (보안상 필요)
    os.chmod(config_path, 0o600)

def local_set():
    print("로컬에서는 이미 expect가 설치되어있어야합니다. 없으면 3번째 줄 주석하시고 local_set() 함수 실행")
    os.system("dnf install -y expect")

#local_ssh_config()
#local_set()

